<a href="https://colab.research.google.com/github/aswinaus/ML/blob/main/Azure_DocumentIntelligence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install azure-ai-formrecognizer openai

In [9]:
# Step 1: Parse document using Azure Document Intelligence
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
from openai import OpenAI

# Replace with your actual access token
from google.colab import userdata
DOCUMENTINTEL_KEY = userdata.get('DOCUMENTINTEL_KEY')

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import nest_asyncio
nest_asyncio.apply()


from google.colab import drive
drive.mount('/content/drive')

data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

# Azure Document Intelligence setup
endpoint = "https://documentsclassifier.cognitiveservices.azure.com/"
key = documentintelligence_token

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(DOCUMENTINTEL_KEY)
)

# Analyze a document
with open(f"{data_dir}/RAG/data/10k/lyft_10k_2023.pdf", "rb") as f:
    poller = document_analysis_client.begin_analyze_document("prebuilt-document", document=f)
result = poller.result()

# Extract text from document
#extracted_text = "\n".join([page.content for page in result.pages])
extracted_text=result.content

# Step 2: Send extracted content to GPT for classification
client = OpenAI(api_key=OPENAI_API_KEY)

prompt = f"""
Classify the following document text into an appropriate category.
Also return the number of pages.
include the precide page number where the 'Financial and Operational Results for the Year Ended December 31, 2023' is included in the document.
If the above exists in more than one page have it displayed as comma separated like 1,2
Return JSON with fields: category, confidence, description, Number of Pages, Page Number.

Document:
{extracted_text}
"""

response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-4.0"
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
```json
{
  "category": "Financial Report",
  "confidence": 0.95,
  "description": "This document is a Form 10-K, an annual report filed by Lyft, Inc. with the United States Securities and Exchange Commission. It provides a comprehensive overview of the company's business operations, risk factors, financial condition, and results of operations for the fiscal year ended December 31, 2023.",
  "Number of Pages": 2,
  "Page Number": "55, 73"
}
```
